Задача:
Выполнить оценку работы библиотеки по извлечению текста новости. Необходимо определить полноту извлекаемого текста и оценить значимость тех частей текста, которые выделить не удалось. Проанализировать и предложить изменения в алгоритм работы библиотеки.
 
Желаемый результат:
Заполненная таблица -
с эталоном текста для каждой новости;
с результатами оценки полноты (сравнения эталона текста и текста, извлеченного библиотекой);
оценка значимости текста, который выделить не удалось;
комментарии, если есть предложения по улучшению.

In [1]:
# 📦 Стандартные библиотеки
import os
import re
from typing import List
from operator import itemgetter

# for .env file support and get access to api keys
from dotenv import load_dotenv
load_dotenv()

# 📦 Сторонние библиотеки
import pandas as pd
from pydantic import BaseModel, Field

# 🚀 LangChain и расширения
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import (
    PydanticOutputParser,
    StructuredOutputParser,
    OutputFixingParser,
)
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.runnables import (
    Runnable,
    RunnablePassthrough,
    chain,
    RunnableConfig,
)

# 🔎 Graph Retriever
from langchain_graph_retriever import GraphRetriever
from graph_retriever.strategies import Eager
from langchain.schema import Document  # Если нужен именно этот Document
from dotenv import load_dotenv
import glob
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_deepseek import ChatDeepSeek




In [ ]:
# import pandas as pd

# from pydantic import BaseModel, Field
# from typing import Dict, Any, List, Optional

# from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
# from langchain.output_parsers import OutputFixingParser
# from langchain_core.tools import tool
# from langchain_core.runnables import RunnablePassthrough
# from langchain.agents import initialize_agent, AgentType


# from langchain_huggingface.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# gpu_llm = HuggingFacePipeline.from_model_id(
#     model_id="../models/Mistral-7B-Instruct-v0.3",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 50},
# )


# cpu_llm = HuggingFacePipeline.from_model_id(
#     model_id="../models/DeepSeek-R1-Distill-Qwen-7B",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 50},
# )


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [2]:

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)


gpu_chain = prompt | gpu_llm

question = "What is electroencephalography?"

print(gpu_chain.invoke({"question": question}))

Question: What is electroencephalography?

Answer: Let's think step by step.

Electroencephalography (EEG) is a technique used to record the electrical activity of the brain.

Here's a breakdown:

1. 'Electro': Related to electricity.

2


In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

cpu_chain = prompt | cpu_llm

question = "What is electroencephalography?"

print(cpu_chain.invoke({"question": question}))

Question: What is electroencephalography?

Answer: Let's think step by step. Electroencephalography, or EEG, is a method used to record electrical activity in the brain. It works by placing small sensors, called electrodes, on the scalp. These electrodes detect the electrical signals produced by the brain's neurons as they communicate


## Load data

In [2]:
data_test_web = pd.read_csv("tmp/data_test_web.csv")
data_test_web = data_test_web.dropna(subset=["web_text"]).reset_index(drop=True)
data_test_web.head()

,URL,lib_text,web_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,Серьезнее других от введения западных санкций ...
1,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,Со следующего года начнется реализация 12 мега...
2,https://expert.ru/finance/investorov-svyazyvay...,"Говоря сухим языком определений, цифровые фина...","Говоря сухим языком определений, цифровые фина..."
3,https://expert.ru/mnenie/vse-teper-zavisit-tol...,— За последние два с половиной года из-за введ...,— За последние два с половиной года из-за введ...
4,https://expert.ru/news/rynok-onlayn-torgovli-v...,"По данным INFOLine, на e-com в 2024 году пришл...","По данным INFOLine, на e-com в 2024 году пришл..."


In [3]:
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [13]:
llm.invoke('hello, how are you?')

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today? 😊", additional_kwargs={'refusal': None, 'reasoning_content': 'Okay, the user greeted me with "hello, how are you?" I need to respond appropriately. Since I\'m an AI, I don\'t have feelings, but I should acknowledge their greeting and offer help. Let me keep it friendly and open. Maybe say I\'m here to assist and ask how I can help them today. That should cover it without overcomplicating things.'}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 11, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 77, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-reasoner', 'system_fin

In [15]:
class OneComment(BaseModel):
    comment: str = Field(..., description="Выведи комментарий к оценке полноты раскрытия, укажи, что раскрыто полностью, частично или не раскрыто и почему ты так думаешь")
    one_score: str = Field(..., description='''Выведи оценку полноты раскрытия ключевого момента от 0 до 10
                           , где 0 - ключевые моменты отсутствуют и/или не раскрыты, 10 - полностью раскрыты или лучше раскрыты.
                           ''')

class OutputData(BaseModel):
    key_moments: List[OneComment] = Field(..., description="Выведи ключевые моменты раскрытия темы, которые ты будешь оценивать")
    comment: str = Field(..., description="Выведи комментарий к средней оценки полноты раскрытия по всем ключевым моментам")
    overall_score: int = Field(..., description='''Выведи среднюю оценку полноты раскрытия по всем ключевым моментам от 0 до 10
                               , где 0 - ключевые моменты отсутствуют и/или не раскрыты, 10 - полностью раскрыты или лучше раскрыты.
                           ''')

parser = PydanticOutputParser(pydantic_object=OutputData)
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    '''
    Выделить ключевые моменты текста {web_text}. Определить наличие этих ключевых моментов и полноту их раскрытия от 0 до 10 в тексте {lib_text}, 
    где 0 - ключевые моменты отсутствуют и/или не раскрыты, 10 - полностью раскрыты или лучше раскрыты.
    Формируй вывод следующим образом:{template}
'''
)

llm_chain = (
    prompt
    | llm 
    | parser
)

result = llm_chain.invoke({
    'template': parser.get_format_instructions(),
    "web_text": data_test_web["web_text"][0],
    "lib_text": data_test_web["lib_text"][0]
})

result

OutputData(key_moments=[OneComment(comment='Уязвимость импортозависимых отраслей (нефтегаз, машиностроение, авиапром) раскрыта полностью: приведены статистика (48.2% в фармацевтике) и примеры компаний.', one_score='10'), OneComment(comment='Переход на китайские технологии в автопроме рассмотрен частично: упомянуты кейсы («АвтоВАЗ»), но отсутствует анализ долгосрочных последствий.', one_score='7'), OneComment(comment='Развитие МСП раскрыто поверхностно: упомянута роль, но нет конкретных механизмов или данных.', one_score='6'), OneComment(comment='Адаптация финансовой инфраструктуры (СПФС, цифровой рубль) описана неполно: глобальные альтернативы SWIFT не проанализированы.', one_score='7'), OneComment(comment='Противоречивые последствия снятия санкций раскрыты хорошо: показаны риски для производителей и исторические прецеденты (Куба, Иран).', one_score='9')], comment='Текст охватывает большинство ключевых аспектов с опорой на данные и примеры, но отдельные моменты (ВПК, долгосрочные прогн

In [17]:
class AspectOneKeyMoment(BaseModel):
    aspect_key_moment: str = Field(..., description="Выведи аспект ключевого момента")
    aspect_key_moment_comment: str = Field(..., description="Выведи подробный комментарий к аспекту ключевого момента")

class OneKeyMoment(BaseModel):
    # comment: str = Field(..., description="Выведи подробное описанием ключевого момента.")
    one_key_moment: str = Field(..., description='''
                            Выведи ключевой момент.
                           ''')
    all_aspects: List[AspectOneKeyMoment] = Field(..., description="Выведи все аспекты ключевого момента")
    one_key_moment_comment: str = Field(..., description="Выведи комментарий по ключевому моменту")


class AllKeyMoments(BaseModel):
    all_key_moments: List[OneKeyMoment] = Field(..., description="Выведи все ключевые моменты")
    all_key_moments_comment: str = Field(..., description="Выведи комментарий по всем ключевым моментам")


parser = PydanticOutputParser(pydantic_object=AllKeyMoments)
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
prompt_key_points = ChatPromptTemplate.from_template(
    '''
    Выделить ключевые моменты текста {web_text} с подробным описанием каждого ключевого момента.
    Отметь подробно все важные аспекты каждого ключевого момента.
    Формируй вывод следующим образом:{template}
'''
)

prompt_score = ChatPromptTemplate.from_template(
    '''
    Определи в тексте {lib_text} наличие ключевых моментов, аспектов ключевых моментов и их подробное описание из {key_moments_web}.
    Формат поступления ключевых моментов, аспектов ключевых моментов и их подробного описания: {template}.
    Формат вывода:
      и полноту их раскрытия от 0 до 10 в тексте {lib_text}, 
    где 0 - ключевые моменты отсутствуют и/или не раскрыты, 10 - полностью раскрыты или лучше раскрыты.
'''
)

llm_chain = (
    prompt_key_points
    | llm 
    | parser

)

result = llm_chain.invoke({
    'template': parser.get_format_instructions(),
    "web_text": data_test_web["web_text"][0],
    "lib_text": data_test_web["lib_text"][0]
})

result

AllKeyMoments(all_key_moments=[OneKeyMoment(one_key_moment='Влияние санкций на ключевые отрасли экономики России', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Наиболее пострадавшие отрасли', aspect_key_moment_comment='Нефтегазовый сектор, машиностроение, авиастроение, фармацевтика, электроника и IT. Санкции ограничили доступ к технологиям и сервисам, что привело к необходимости импортозамещения.'), AspectOneKeyMoment(aspect_key_moment='Прогнозы аналитиков ЦМАКП', aspect_key_moment_comment='В марте 2022 г. прогнозировалось наибольшее влияние на фармацевтику (48.2%), химическую промышленность (44.7%) и авиастроение (32.2%).')], one_key_moment_comment='Санкции стимулировали развитие отечественного производства, но в ряде отраслей (станкостроение, судостроение) замещение импорта остается незавершенным.'), OneKeyMoment(one_key_moment='Сценарии снятия санкций США', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Условия отмены', aspect_key_moment_comment='США снимают санкции только

In [20]:
result.all_key_moments

[OneKeyMoment(one_key_moment='Влияние санкций на ключевые отрасли экономики России', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Наиболее пострадавшие отрасли', aspect_key_moment_comment='Нефтегазовый сектор, машиностроение, авиастроение, фармацевтика, электроника и IT. Санкции ограничили доступ к технологиям и сервисам, что привело к необходимости импортозамещения.'), AspectOneKeyMoment(aspect_key_moment='Прогнозы аналитиков ЦМАКП', aspect_key_moment_comment='В марте 2022 г. прогнозировалось наибольшее влияние на фармацевтику (48.2%), химическую промышленность (44.7%) и авиастроение (32.2%).')], one_key_moment_comment='Санкции стимулировали развитие отечественного производства, но в ряде отраслей (станкостроение, судостроение) замещение импорта остается незавершенным.'),
 OneKeyMoment(one_key_moment='Сценарии снятия санкций США', all_aspects=[AspectOneKeyMoment(aspect_key_moment='Условия отмены', aspect_key_moment_comment='США снимают санкции только при достижении целей их введ

In [33]:
rows = []
for n, item in enumerate(result.all_key_moments):
    for aspect in item.all_aspects:
        rows.append({
            'key_moment_id': n,
            "key_moment": item.one_key_moment,
            "key_moment_comment": item.one_key_moment_comment,
            "aspect": aspect.aspect_key_moment,
            "aspect_comment": aspect.aspect_key_moment_comment
        })

df = pd.DataFrame(rows)

df

,key_moment_id,key_moment,key_moment_comment,aspect,aspect_comment
0,0,Влияние санкций на ключевые отрасли экономики ...,Санкции стимулировали развитие отечественного ...,Наиболее пострадавшие отрасли,"Нефтегазовый сектор, машиностроение, авиастрое..."
1,0,Влияние санкций на ключевые отрасли экономики ...,Санкции стимулировали развитие отечественного ...,Прогнозы аналитиков ЦМАКП,В марте 2022 г. прогнозировалось наибольшее вл...
2,1,Сценарии снятия санкций США,Полная отмена маловероятна; возможны точечные ...,Условия отмены,США снимают санкции только при достижении целе...
3,1,Сценарии снятия санкций США,Полная отмена маловероятна; возможны точечные ...,Политическая мотивация,Санкции — инструмент давления. Их смягчение во...
4,2,Трансформация автопрома,Рынок авто стагнирует из-за высоких кредитных ...,Переход на китайские платформы,"Заводы в Калининграде, Москве и Калуге перешли..."
5,2,Трансформация автопрома,Рынок авто стагнирует из-за высоких кредитных ...,Риски возврата западных брендов,Возврат иностранных производителей потребует у...
6,3,Авиапром и импортозамещение,Авиакомпании инвестируют в российское ПО и зап...,100% локализация,Планы по выпуску отечественных самолетов к 202...
7,3,Авиапром и импортозамещение,Авиакомпании инвестируют в российское ПО и зап...,Санкционные риски,"США могут смягчить санкции в обмен на уступки,..."
8,4,Роль МСП в импортозамещении,"МСП стали драйвером замещения, но зависимость ...",Рост числа предприятий,С 5.8 млн (2022 г.) до 6.3 млн (2023 г.) — мал...
9,4,Роль МСП в импортозамещении,"МСП стали драйвером замещения, но зависимость ...",Китайская конкуренция,В легкой промышленности китайские производител...
